In [2]:
# Process a single page from the PDF:
# 1. Convert PDF page to image
# 2. Extract text using OCR
# 3. Convert to markdown format
# 4. Use GPT to clean up OCR mistakes and improve formatting
# 5. Save the final markdown to a file

from pdf2image import convert_from_path
import pytesseract
import os
import openai

def setup_paths():
    """Set up poppler and tesseract paths based on system architecture"""
    if os.path.exists('/opt/homebrew/bin'):  # For Apple Silicon Macs
        poppler_path = '/opt/homebrew/bin'
        pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'
    elif os.path.exists('/usr/local/bin'):   # For Intel Macs
        poppler_path = '/usr/local/bin'
        pytesseract.pytesseract.tesseract_cmd = '/usr/local/bin/tesseract'
    else:
        raise Exception("Couldn't find required paths")
        
    print(f"Using poppler path: {poppler_path}")
    print(f"Using tesseract path: {pytesseract.pytesseract.tesseract_cmd}")
    return poppler_path

def convert_pdf_to_image(pdf_path, poppler_path, first_page=None, last_page=None, dpi=300):
    """Convert PDF pages to images"""
    pages = convert_from_path(
        pdf_path,
        first_page=first_page,
        last_page=last_page,
        poppler_path=poppler_path,
        dpi=dpi
    )
    return pages[0]

def preprocess_image(image, threshold=200):
    """Preprocess image for better OCR results"""
    image = image.convert('L')  # Convert to grayscale
    return image.point(lambda x: 0 if x < threshold else 255, '1')

def extract_text_from_image(image):
    """Extract text from image using OCR"""
    custom_config = r'--oem 3 --psm 3'
    return pytesseract.image_to_string(image, config=custom_config)

def is_likely_header(line, header_type='h1'):
    """Determine if a line is likely a header based on characteristics"""
    common_words = ['the', 'and', 'or', 'but', 'in', 'on', 'at']
    
    if header_type == 'h1':
        return (len(line) <= 50 and 
                line.isupper() and 
                not any(char.isdigit() for char in line) and
                not any(word in line.lower() for word in common_words) and
                not ('[' in line or ']' in line))
    elif header_type == 'h2':
        return (len(line) <= 70 and 
                line[0].isupper() and 
                line[-1] not in '.?!' and 
                not any(char.isdigit() for char in line) and
                not any(word in line.lower() for word in common_words) and
                not ('[' in line or ']' in line))
    elif header_type == 'h3':
        return len(line) <= 100 and line.endswith(':') and not ('[' in line or ']' in line)
    
    return False

def convert_to_markdown(text):
    """Convert extracted text to markdown format"""
    lines = text.split('\n')
    markdown_text = ""
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        if is_likely_header(line, 'h1'):
            markdown_text += f"# {line.title()}\n\n"
        elif is_likely_header(line, 'h2'):
            markdown_text += f"## {line}\n\n"
        elif is_likely_header(line, 'h3'):
            markdown_text += f"### {line[:-1]}\n\n"
        else:
            markdown_text += f"{line}\n\n"
            
    return markdown_text

def correct_markdown_with_gpt(markdown_text, client):
    """Use GPT to correct OCR mistakes and improve markdown formatting"""
    messages = [
        {"role": "system", "content": "You are a helpful assistant that fixes OCR mistakes and formats text as proper markdown. Pay special attention to bullet points, ensuring they use proper markdown syntax (- or *). Also fix any obvious OCR errors while preserving the original meaning."},
        {"role": "user", "content": f"Please fix any OCR mistakes in this text and format it as proper markdown:\n\n{markdown_text}"}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.3,
        max_tokens=1500
    )
    
    return response.choices[0].message.content

def main():
    client = openai.OpenAI(
        api_key=os.getenv('OPENAI_TOKEN'),
    )

    try:
        poppler_path = setup_paths()
        
        # Process PDF page
        page_image = convert_pdf_to_image('Atomic habits.pdf', poppler_path, first_page=68, last_page=68)
        processed_image = preprocess_image(page_image)
        extracted_text = extract_text_from_image(processed_image)
        
        # Convert to markdown and correct with GPT
        markdown_text = convert_to_markdown(extracted_text)
        corrected_markdown = correct_markdown_with_gpt(markdown_text, client)
        
        # Save results
        with open('Atomic habits test.md', 'w', encoding='utf-8') as f:
            f.write(corrected_markdown)
        
        print(f"Successfully saved corrected markdown to 'Atomic habits test.md'")
        print(f"Number of lines processed: {len(extracted_text.split('\n'))}")

    except FileNotFoundError:
        print("Error: Could not find 'Atomic habits.pdf'")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Using poppler path: /opt/homebrew/bin
Using tesseract path: /opt/homebrew/bin/tesseract
Successfully saved corrected markdown to 'Atomic habits test.md'
Number of lines processed: 18


In [5]:
# Process the entire book page by page:
# 1. Convert PDF pages to images
# 2. Extract text using OCR
# 3. Convert to markdown format
# 4. Use GPT to clean up OCR mistakes and improve formatting
# 5. Save the final markdown to a single file
from tqdm import tqdm
import os
import openai

try:
    # Convert all pages of PDF to images
    pages = convert_from_path(
        'Atomic habits.pdf',
        poppler_path = setup_paths()
    )
    
    print(f"Total pages found: {len(pages)}")
    
    client = openai.OpenAI(
        api_key=os.getenv('OPENAI_TOKEN'),
    )
    
    # Process each page
    full_markdown = ""
    for page in tqdm(pages, desc="Processing pages"):
        # Extract text from page
        processed_image = preprocess_image(page)
        extracted_text = extract_text_from_image(processed_image)
        
        # Convert to markdown and correct with GPT
        markdown_text = convert_to_markdown(extracted_text)
        corrected_markdown = correct_markdown_with_gpt(markdown_text, client)
        
        full_markdown += corrected_markdown + "\n\n"  # Add extra newlines between pages
        
    # Save final markdown
    with open('Atomic habits.md', 'w', encoding='utf-8') as f:
        f.write(full_markdown)
        
    print(f"Successfully saved markdown to Atomic habits.md")
    print(f"Total characters: {len(full_markdown)}")

except Exception as e:
    print(f"An error occurred: {str(e)}")


Using poppler path: /opt/homebrew/bin
Using tesseract path: /opt/homebrew/bin/tesseract
Total pages found: 256


Processing pages: 100%|██████████| 256/256 [23:17<00:00,  5.46s/it]

Successfully saved markdown to Atomic habits.md
Total characters: 434107


In [1]:
# Read the context file
with open('my context.txt', 'r', encoding='utf-8') as f:
    context = f.read()

# Read the book summary
with open('book summary.txt', 'r', encoding='utf-8') as f:
    summary = f.read()

prompt_prefix = f"""
HERE IS A SUMMARY OF THE ATOMIC HABITS BOOK BY JAMES CLEAR:
{summary}
END OF SUMMARY.

HERE ARE THE CHAPTERS OF THE BOOK:
The Fundamentals
Why Tiny Changes Make a Big Difference
1 The Surprising Power of Atomic Habits
2 How Your Habits Shape Your Identity (and Vice Versa) 
3 How to Build Better Habits in 4 Simple Steps

The 1st Law Make It Obvious
4 The Man Who Didn’t Look Right
5 The Best Way to Start a New Habit
6 Motivation Is Overrated; Environment Often Matters More
7 The Secret to Self-Control

The 2nd Law Make It Attractive
8 How to Make a Habit Irresistible
9 The Role of Family and Friends in Shaping Your Habits
10 How to Find and Fix the Causes of Your Bad Habits

The 3rd Law Make It Easy
11 Walk Slowly, but Never Backward
12 The Law of Least Effort
13 How to Stop Procrastinating by Using the Two-Minute Rule
14 How to Make Good Habits Inevitable and Bad Habits Impossible

The 4th Law Make It Satisfying
15 The Cardinal Rule of Behavior Change
16 How to Stick with Good Habits Every Day
17 How an Accountability Partner Can Change Everything

Advanced Tactics How to Go from Being Merely Good to Being Truly Great
18 The Truth About Talent (When Genes Matter and When They Don’t)
19 The Goldilocks Rule: How to Stay Motivated in Life and Work
20 The Downside of Creating Good Habits
END OF CHAPTERS.

HERE IS MY CONTEXT:
{context}
END OF CONTEXT.
"""


def get_prompt_suffix(chunk):
    prompt_suffix = f"""
    HERE IS THE PART OF THE BOOK IN THE MARKDOWN FORMAT:
    {chunk}
    END OF THE BOOK PART.

    Please rewrite this part of the book in a way that is a bit more personalized to my own life (use the CONTEXT provided). You should not change chapter names, or anything high-level, you should leave 80-90% of the text untouched. You should only add examples or analogies about my own life, how the principles described in the book are relevant to me.
    Keep the same formatting as in the original part of the book. Don't write anything else except the rewritten text. Your work:"""
    return prompt_suffix

In [13]:
# This cell processes the Atomic Habits book text by splitting it into chunks,
# sending each chunk to GPT-4 for personalization based on user context,
# and saving the rewritten version to a new file

import time

# Read the original cleaned text
with open('Atomic habits.md', 'r', encoding='utf-8') as f:
    text = f.read()

# Split into chunks of ~2000 characters
chunk_size = 2000
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Process each chunk with ChatGPT
rewritten_chunks = []
for chunk in tqdm(chunks, desc="Processing chunks"):
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are James Clear, author of the Atomic Habits book. You are rewriting the book for a specific person based on their context."},
                {"role": "user", "content": prompt_prefix + get_prompt_suffix(chunk)}
            ]
        )
        rewritten_chunks.append(response.choices[0].message.content)
        time.sleep(1) # Rate limiting
    except Exception as e:
        print(f"Error processing chunk: {str(e)}")
        rewritten_chunks.append(chunk) # Keep original on error
        
# Combine chunks and save to new file
rewritten_text = "\n".join(rewritten_chunks)

with open('Atomic habits rewritten v1.md', 'w', encoding='utf-8') as f:
    f.write(rewritten_text)

print(f"Successfully saved rewritten text to Atomic habits rewritten.md")
print(f"Original length: {len(text)}")
print(f"Rewritten length: {len(rewritten_text)}")

Processing chunks: 100%|██████████| 161/161 [40:41<00:00, 15.16s/it]

Successfully saved rewritten text to Atomic habits rewritten.md
Original length: 320762
Rewritten length: 453315


In [2]:
import re
from tqdm import tqdm
import openai
import os

input_file = 'Atomic habits.md'
output_file = 'Atomic habits personalized summaries v2.md'
summary_length = 2000  # Approximate characters for summary

client = openai.OpenAI(
    api_key=os.getenv('OPENAI_TOKEN'),
)

def split_into_chapters(text):
    """
    Split text into chapters based on '#' and '## ' headers.
    If a '##' header is the first after a '#', include it in the current chapter.
    Finish chapters based on the header level encountered.
    """
    chapters = []
    current_chapter = []
    lines = text.split('\n')
    
    previous_header_level = None

    for line in lines:
        header_match = re.match(r'^(#{1,2})\s+(.*)', line)
        if header_match:
            header_level = len(header_match.group(1))
            header_text = header_match.group(2).strip()
            if previous_header_level is None:
                # Start the first chapter
                current_chapter.append(line)
            else:
                if header_level <= previous_header_level:
                    # Finish current chapter and start a new one
                    chapters.append('\n'.join(current_chapter))
                    current_chapter = [line]
                else:
                    # Continue current chapter
                    current_chapter.append(line)
            previous_header_level = header_level
        else:
            if current_chapter:
                current_chapter.append(line)
    if current_chapter:
        chapters.append('\n'.join(current_chapter))
    return chapters

def generate_summary(text):
    # Call GPT-4 to generate a summary
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that summarizes text into a concise format."
                },
                {
                    "role": "user",
                    "content": f"Please summarize the following text in approximately two A4 pages:\n\n{text}"
                }
            ]
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        print(f"Error generating summary: {str(e)}")
        return ""

def generate_personalized_summary_and_actionables(chapter_title, chapter_text, cumulative_summary, user_context):
    # Call GPT-4 to generate personalized summary and actionable items
    try:
        prompt = (
            f"INSTRUCTION:\nI will give you 1) a cumulative summary of the previous chapters of the book Atomic Habits, 2) along with the new chapter titled '{chapter_title}', 3) along with my own context, "
            f"please provide a PERSONALIZED SPECIFICALLY TO ME SUMMARY of this chapter and at least 20 actionable, clear ideas on how I can apply the principles from this chapter to their life, be very specific, NO FLUFF.\n\n"
            f"CUMULATIVE SUMMARY:\n{cumulative_summary}\n\n"
            f"CHAPTER TITLE:\n{chapter_title}"
            f"CHAPTER TEXT:\n{chapter_text}\n\n"
            f"MY CONTEXT:\n{user_context}\n\n"
            f"INSTRUCTION:\nNow you have 1) a cumulative summary of the previous chapters of the book Atomic Habits, 2) along with the new chapter titled '{chapter_title}', 3) along with my own context, "
            f"please provide a PERSONALIZED SPECIFICALLY TO ME SUMMARY of this chapter and 20 actionable, clear ideas on how I can apply the principles FROM THIS CHAPTER (not the whole book) to my life, be very specific, NO FLUFF.\n"
            "ADD EMOJIS to the output to make it more engaging. Please provide the output in the following Markdown structure:\n"
            "### SUMMARY\nsumary text\n"
            "### ACTIONABLES\n1. Short name of the idea + emoji: text of the idea\n2. ...\nYOUR WORK:"
        )
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a personal growth coach of billionaire CEO's. You are helping them to improve their life by changing their habits."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        result = response.choices[0].message.content.strip()
        return result
    except Exception as e:
        print(f"Error generating personalized summary and actionables: {str(e)}")
        return ""

def get_user_context():
    with open('my context.txt', 'r', encoding='utf-8') as f:
        context = f.read()
    return context.strip()

# Read the original book text
with open(input_file, 'r', encoding='utf-8') as f:
    text = f.read()

# Split the text into chapters
chapters = split_into_chapters(text)

cumulative_summary = ""
output_content = "# Personalized Summaries of Atomic Habits\n\n"

user_context = get_user_context()

for chapter in tqdm(chapters, desc="Processing chapters"):
    # Extract chapter title and text
    lines = chapter.strip().split('\n', 1)
    if len(lines) < 2:
        continue  # Skip if chapter format is unexpected
    chapter_title = lines[0].strip()
    chapter_text = lines[1].strip()

    # Update cumulative summary
    if cumulative_summary:
        combined_text = cumulative_summary + "\n\n" + chapter_text
    else:
        combined_text = chapter_text
    cumulative_summary = generate_summary(combined_text)

    # Generate personalized summary and actionables
    personalized_output = generate_personalized_summary_and_actionables(
        chapter_title, chapter_text, cumulative_summary, user_context
    )

    # Append to output content with proper Markdown structure
    output_content += f"## {chapter_title}\n\n"
    output_content += f"{personalized_output}\n\n"
    print(f"{chapter_title}\n{personalized_output}\n\n")

# Save the personalized summaries to a new MD file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_content)

print(f"Successfully saved personalized summaries to {output_file}")


Processing chapters:   5%|▍         | 1/21 [00:41<13:55, 41.80s/it]

# Fundamentals: Why Tiny Changes Make a Big Difference
### SUMMARY
This chapter of "Atomic Habits" emphasizes the transformative power of minuscule changes and their compounding effects over time. Inspired by the British Cycling team's remarkable success through marginal gains, it teaches that a series of tiny, seemingly insignificant actions can lead to massive results. The focus shifts from lofty goals to consistent, incremental improvements - typical 1% betterment each day - that in their totality, become a catalyst for significant personal and professional growth. Your ambition to enhance productivity, organizational skills, and overall health aligns with the concept of leveraging small habitual changes for larger life impacts.

### ACTIONABLES
1. **Minute Morning Review 🌄**: Spend the first 5 minutes after waking up to quickly review your main tasks for the day. This aligns your mind with your daily goals without consuming much time.

2. **Daily Goals Journaling 📔**: Each morning,

Processing chapters:  10%|▉         | 2/21 [01:28<14:04, 44.43s/it]

## How Your Habits Shape Your Identity (and Vice Versa)
### SUMMARY
The chapter on "How Your Habits Shape Your Identity (and Vice Versa)" emphasizes the profound impact our daily practices have on who we are and who we become. Focusing on identity-based habits rather than just outcome-based goals can help ensure that changes in behavior are sustainable and aligned with who you aspire to be. By affirming and acting on the type of person you want to become, each habit you cultivate serves as a vote toward this identity. This approach not only aids in habit formation but also in alleviating conflicts between your goals and your sense of self, facilitating a more authentic and integrated personal growth.

### ACTIONABLES
1. **Identify Your Desired Identity 🎯**: Reflect deeply about the persona you aspire to become. Considering your current goals, define yourself as someone who embodies the qualities needed to achieve them. For example, "I am a disciplined and focused professional."

2. **V

Processing chapters:  14%|█▍        | 3/21 [02:11<13:10, 43.92s/it]

...

## The Goldilocks Rule: How to Stay Motivated in Life and Work
### SUMMARY
The "Goldilocks Rule" 🐻📏 emphasizes that peak motivation and engagement in habits occur when tasks challenge us just enough—not too easy to be boring, yet not so difficult that they feel unachievable. Applying this principle means adjusting your tasks and goals to hit this sweet spot, maintaining a balance that keeps tasks engaging without being overwhelming. For you, an ambitious software engineer, mastering the delicate art of balancing your work and personal growth activities is key. Focusing on optimizing your productivity levels and integrating new learning habits will help you maintain a high level of motivation and satisfaction.

### ACTIONABLES
1. **Adjust task difficulty + ⚖️**: Tailor your work tasks daily to ensure they're not overly simple or mind-numbing but also not excessively challenging. Adjust the complexity based on your current limits.

2. **Track progress visually + 📊**: Use visual tracking 

Processing chapters:  95%|█████████▌| 20/21 [14:50<00:44, 44.67s/it]

## The Downside of Creating Good Habits
### SUMMARY
In the latest chapter, "The Downside of Creating Good Habits," the primary focus shifts to recognizing and overcoming the limitations that well-ingrained habits can introduce, especially when they become so automatic that they stymie innovation and personal growth. The key takeaway is that while habits form the backbone of productivity and success, there's a crucial need for deliberate practice and continuous reflection to ensure these habits don't make you complacent. This is particularly relevant to your life as a young software engineer looking to balance high productivity with personal well-being. You aim for not only efficiency but mastery and growth in both professional and personal spheres.

### ACTIONABLES
1. **Schedule Deliberate Practice Sessions 🎯**: Allocate specific times in your week dedicated solely to honing and critiquing your coding skills beyond daily tasks, focusing on unfamiliar or challenging projects.

2. **Mont

Processing chapters: 100%|██████████| 21/21 [15:39<00:00, 44.72s/it]

## Conclusion: The Secret to Results That Last
### SUMMARY
The "Conclusion: The Secret to Results That Last" chapter of "Atomic Habits" by James Clear encapsulates the essence of forming and maintaining successful habits through the analogy of the Sorites Paradox, emphasizing the compounding effect of small, consistent improvements. By iteratively applying the Four Laws of Behavior Change—Make it Obvious, Attractive, Easy, and Satisfying—Clear proposes a sustainable model to gradually tilt life's scales in your favor. This final chapter is a compelling reminder that success isn’t achieved by leaps, but through incremental, relentless steps forward, making slightly better decisions day by day. For you, a young software engineer with robust personal and professional goals, this chapter reinforces the necessity to cultivate a system conducive to continuous improvement without wavering in the face of routine challenges.

### ACTIONABLES
1. **Tiny Morning Rituals ☀️**: Start each day by set